In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_lg')

In [8]:
text = 'this is ruban did you know him'
doc = nlp(text)
for i in doc:
    print(f"vector: {i.has_vector}, i, out_of_voc {i.is_oov}")

vector: True, i, out_of_voc False
vector: True, i, out_of_voc False
vector: True, i, out_of_voc False
vector: True, i, out_of_voc False
vector: True, i, out_of_voc False
vector: True, i, out_of_voc False
vector: True, i, out_of_voc False


In [11]:
doc[2].vector.shape

(300,)

In [14]:
nlp("look").similarity(nlp('see'))

0.7343267798423767

In [21]:
li = ['car', 'bike', 'cat', 'tiger']
doc = nlp(" ".join(li))
for i in doc:
    print(i.text, i.similarity(nlp('lion')))

car 0.1752486228942871
bike 0.13621169328689575
cat 0.5265437960624695
tiger 0.7359829545021057


In [26]:
def print_similarity(base, text):
    base = nlp(base)
    doc = nlp(text)

    similar = []
    for i in doc:
        similar.append(f'{i.text} <=> {base} ==>{i.similarity(base)}')
    return similar

In [30]:
print_similarity("iphone", "Camera quality is good")

['Camera <=> iphone ==>0.4052708148956299',
 'quality <=> iphone ==>0.2035873532295227',
 'is <=> iphone ==>0.14933013916015625',
 'good <=> iphone ==>0.20680975914001465']

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
king = nlp('king').vector
man = nlp('man').vector

In [37]:
cosine_similarity([king], [man])

array([[0.40884614]], dtype=float32)

# News classification


In [38]:
import pandas as pd

In [39]:
df = pd.read_csv(r"C:\Users\KIRUBA\Downloads\Fake.csv\Fake.csv")

In [40]:
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [41]:
df1 = pd.read_csv(r"C:\Users\KIRUBA\Downloads\True.csv\True.csv")

In [42]:
df1.head(2)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"


In [48]:
df.shape

(23481, 4)

In [49]:
df1.shape

(21417, 4)

In [50]:
df['subject num'] = 1 

In [53]:
df1['subject num'] = 0

In [54]:
df_fake = df.sample(5000, random_state=5000)
df_real = df1.sample(5000, random_state=5000)

In [55]:
df2 = pd.concat([df_fake, df_real], axis = 'rows', ignore_index=True)

In [56]:
df2.sample(4)

,title,text,subject,date,subject num
417,HOW MEXICO COULD ACTUALLY Benefit From A Trump...,It s easier to hate Trump and spew venom at hi...,politics,"Apr 25, 2016",1
243,"BLACK FELON Brutally Beats Girlfriend, Grabs C...",Another case of the White man and an oppressiv...,politics,"Mar 31, 2016",1
6648,Trump seeks $10 million from former adviser in...,(Reuters) - Republican White House hopeful Don...,politicsNews,"July 13, 2016",0
5383,Austria's leaders reject Juncker's vision for ...,VIENNA (Reuters) - Austria s Social Democrat C...,worldnews,"September 14, 2017",0


In [57]:
df2['subject num'].value_counts()

subject num
1    5000
0    5000
Name: count, dtype: int64

In [58]:
df2['vector'] = df['text'].apply(lambda x: nlp(x).vector)

In [59]:
df2.sample(2)

,title,text,subject,date,subject num,vector
3117,Samantha Bee Goes There In Funniest NSFW Take...,When it comes to Donald Trump one thing is cle...,News,"April 19, 2016",1,"[-0.02899715, 0.1986675, -0.07658685, -0.03822..."
9772,Slovakia delays decision to replace Russian fi...,BRATISLAVA (Reuters) - Slovakia will postpone ...,worldnews,"September 20, 2017",0,"[-0.04874328, 0.10231716, -0.11242415, -0.0315..."


In [62]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df2['vector'].values, df2['subject num'], test_size=0.2, random_state = 90)

In [65]:
import numpy as np

x_train = np.stack(x_train)
x_test = np.stack(x_test)

In [68]:
x_test.shape

(2000, 300)

In [72]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_tran = scaler.fit_transform(x_train)
x_test_tran = scaler.transform(x_test)

In [73]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train_tran, y_train)

MultinomialNB()

In [74]:
model.score(x_test_tran, y_test)

0.673

In [75]:
from sklearn.metrics import classification_report

In [77]:
print(classification_report(y_test, model.predict(x_test_tran)))

              precision    recall  f1-score   support

           0       0.69      0.64      0.66      1008
           1       0.66      0.71      0.68       992

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000



In [78]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier()
model1.fit(x_train_tran, y_train)

KNeighborsClassifier()

In [80]:
model1.score(x_test_tran, y_test)

0.728

In [82]:
print(classification_report(y_test, model1.predict(x_test_tran)))

              precision    recall  f1-score   support

           0       0.75      0.69      0.72      1008
           1       0.71      0.76      0.74       992

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000



# Exercise

In [83]:
df = pd.read_json(r"C:\Users\KIRUBA\Downloads\News_Category_Dataset.json\News_Category_Dataset.json", lines = True)

In [84]:
df.sample(2)

,short_description,headline,date,link,authors,category
110942,,"DOJ Gets Ferguson, St. Louis County Cops To Ba...",2014-09-26,https://www.huffingtonpost.com/entry/ferguson-...,Ryan J. Reilly,POLITICS
14371,“His siblings are in disbelief over the loss o...,This Syrian Toddler Was Named After His Uncle....,2017-10-11,https://www.huffingtonpost.com/entry/syria-idl...,Rowaida Abdelaziz,WORLD NEWS


In [86]:
df.shape

(124989, 6)

In [87]:
df['category'].value_counts()

category
POLITICS          32739
ENTERTAINMENT     14257
HEALTHY LIVING     6694
QUEER VOICES       4995
BUSINESS           4254
SPORTS             4167
COMEDY             3971
PARENTS            3955
BLACK VOICES       3858
THE WORLDPOST      3664
WOMEN              3490
CRIME              2893
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
IMPACT             2602
WORLDPOST          2579
RELIGION           2556
STYLE              2254
WORLD NEWS         2177
TRAVEL             2145
TASTE              2096
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
SCIENCE            1381
ARTS & CULTURE     1339
TECH               1231
COLLEGE            1144
LATINO VOICES      1129
EDUCATION          1004
Name: count, dtype: int64

In [88]:
df.isnull().sum()

short_description    0
headline             0
date                 0
link                 0
authors              0
category             0
dtype: int64